In [1]:
from sqlanalyzer import column_parser
import pandas as pd
import sqlparse
import re

In [2]:
def delevel_query(query_list):
    line_level = []
    sub_query = []
    pos_delete, pos_where = len(query_list), len(query_list)
    
    for i, line in enumerate(query_list):
        if line.startswith('ORDER') or line.startswith('GROUP'):
            pos_delete = i
        if line.startswith('WHERE'):
            pos_where = i 
        if line.startswith('FROM'):
            pos_from = i-1
#         if line.startswith('LEFT JOIN') or line.startswith('INNER JOIN') or line.startswith('FULL OUTER JOIN'):
#             pos_join = i
  
    for line in query_list[:pos_from+2]:
        line_level.append((line, "level_1"))
        
    for line in query_list[pos_from+1:pos_where]:
        if line.startswith(' ') or line.startswith('FROM'):
            sub_query.append(line[3:])
        else:
            line_level.append((line, "level_1"))

    for line in query_list[pos_where:pos_delete]:    
        line_level.append((line, "level_1"))

    return line_level, sub_query[1:]

In [3]:
def parse_sub_query(sub_query_list):
    sub_query = "\n".join(sub_query_list)
    sub_query = sub_query.lstrip('\n').lstrip(' ')
    
    formatter = column_parser.Parser(sub_query)
    formatted = formatter.format_query(sub_query)
    sub_query_list = formatted.split('\n')

    query = "\n".join(sub_query_list).lstrip('\n').lstrip(' ').lstrip(r'\(')
    query_list = query.split('\n')
    
    return query_list

In [4]:
def has_child(sub_query_list):
    query_list = parse_sub_query(sub_query_list)
    query_list = delevel_query(query_list)[1]
    return query_list, query_list != []

In [5]:
query = """SELECT *
FROM sfdc.accounts sfdc_accounts
LEFT JOIN
  (SELECT MAX(dt)
   FROM
     (SELECT dt
      FROM sfdc.oppty) sfdc_oppty
   LEFT JOIN
     (SELECT dt
      FROM sfdc.cases) ON sfdc_oppty.dt = sfdc_cases.dt) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt
WHERE dt > '2020-04-03' 
"""

In [6]:
formatter = column_parser.Parser(query)
formatted = formatter.format_query(query)
query_list = formatted.split('\n')
# sub_query_list, has_subquery = has_child(query_list)

In [7]:
has_subquery = True
print("True", '\n', formatted, '\n')
while has_subquery:
    sub_query_list, has_subquery = has_child(query_list)
    query_list = sub_query_list
    print(has_subquery, '\n', '\n'.join(query_list), '\n')

True 
 SELECT *
FROM sfdc.accounts sfdc_accounts
LEFT JOIN
  (SELECT MAX(dt)
   FROM
     (SELECT dt
      FROM sfdc.oppty) sfdc_oppty
   LEFT JOIN
     (SELECT dt
      FROM sfdc.cases) ON sfdc_oppty.dt = sfdc_cases.dt) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt
WHERE dt > '2020-04-03' 

True 
 SELECT MAX(dt)
FROM
  (SELECT dt
   FROM sfdc.oppty) sfdc_oppty
LEFT JOIN
  (SELECT dt
   FROM sfdc.cases) ON sfdc_oppty.dt = sfdc_cases.dt) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt 

True 
 SELECT dt
FROM sfdc.oppty) sfdc_oppty
SELECT dt
FROM sfdc.cases) ON sfdc_oppty.dt = sfdc_cases.dt) AS sfdc_cases ON sfdc_cases.dt = sfdc_accounts.dt 

False 
  



## 1-level

In [8]:
query = """SELECT sfdc_oppty.dt, sfdc_cases.id
FROM sfdc.oppty sfdc_oppty
LEFT JOIN sfdc.cases AS sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt
"""

In [9]:
formatter = column_parser.Parser(query)
formatted = formatter.format_query(query)
query_list = formatted.split('\n')
# sub_query_list, has_subquery = has_child(query_list)

In [10]:
has_subquery = True
print("True", '\n', formatted, '\n')
while has_subquery:
    sub_query_list, has_subquery = has_child(query_list)
    query_list = sub_query_list
    print(has_subquery, '\n', '\n'.join(query_list).lstrip('\n'), '\n')

True 
 SELECT sfdc_oppty.dt,
       sfdc_cases.id
FROM sfdc.oppty sfdc_oppty
LEFT JOIN sfdc.cases AS sfdc_cases ON sfdc_oppty.dt = sfdc_cases.dt 

False 
  



In [11]:
formatter._get_all_variables(formatted)

['sfdc_oppty.dt',
 'sfdc_cases.id',
 'sfdc.oppty sfdc_oppty',
 '',
 'sfdc.cases',
 'sfdc_cases',
 'sfdc_oppty.dt',
 'sfdc_cases.dt']

In [12]:
db_fields_1 = pd.DataFrame({'db_table': 'sfdc.oppty', 
            'all_columns': ['platform', 'mobile_os', 'service', 'service_metadata', 'service_metadata_version', 'account', 'num_requests', 'dt']})

db_fields_2 = pd.DataFrame({'db_table': 'sfdc.cases', 
            'all_columns': ['account', 'num_requests', 'dt', 'owner', 'id']})

db_fields = db_fields_1.append(db_fields_2, ignore_index=True)


In [13]:
formatted_dict = {'main': formatted}

In [14]:
formatter._get_all_scanned_cols(formatted_dict, db_fields)

['sfdc.oppty.dt']

In [15]:
# for _,cte_query in cte_queries.items():

table_alias_mapping = formatter.get_table_names(formatted.split('\n'))
variables = formatter._get_all_variables(formatted)
queried_columns = formatter._get_queried_columns(table_alias_mapping, db_fields)
if variables == []:
    original_columns_list = []
    for table in queried_columns:
        for k,v in table.items():
            for t in v:
                original_columns_list.append("{}.{}".format(k,t))
else:
    original_columns_list = formatter._map_db_columns(variables, queried_columns, table_alias_mapping)
    

In [16]:
queried_columns

[{'sfdc.oppty': {'account',
   'dt',
   'mobile_os',
   'num_requests',
   'platform',
   'service',
   'service_metadata',
   'service_metadata_version'}},
 {'sfdc.cases': {'account', 'dt', 'id', 'num_requests', 'owner'}}]

In [17]:
variables

['sfdc_oppty.dt',
 'sfdc_cases.id',
 'sfdc.oppty sfdc_oppty',
 '',
 'sfdc.cases',
 'sfdc_cases',
 'sfdc_oppty.dt',
 'sfdc_cases.dt']

In [18]:
var_list = variables
queried_cols = queried_columns


In [19]:
# def _map_db_columns(self, var_list, queried_cols, table_alias_mapping):
#         """
#         Map database columns.
#         Args:
#             var_list (list): The list of all variables (non-sql reserved words) in query.
#             queried_cols (list): The list of all currently existing columns in Glue, under the table that was being queried. 
#             table_alias_mapping (dict): The mapping of table and their (if) alias.
#         Returns:
#             list: A list of unique db.table.column that was being scanned by the query.
#         """
#         original_columns_list = []

for var in set(var_list):

    var = var.strip(' ')

    if var in table_alias_mapping.keys():
        pass

    else:
        var_split = var.split('.')

        if len(var_split) == 1:
            for db_table in queried_cols:
                for k,v in db_table.items():
                    if var in v:
                        original_columns_list.append("{}.{}".format(k, var))

        elif len(var_split) == 2:

            if var_split[0] in table_alias_mapping.keys():
                db_table = table_alias_mapping[var_split[0]]

                for db_table_col in queried_cols:
                    for k,v in db_table_col.items():
                        if k == db_table and var_split[1] in v:
                            original_columns_list.append("{}.{}".format(k, var_split[1]))

#         return list(set(original_columns_list))


In [20]:
for var in set(var_list):

    var = var.strip(' ')
    print(var)

    if var in table_alias_mapping.keys():
        pass

    else:
        var_split = var.split('.')


sfdc_cases.id
sfdc_cases.dt
sfdc.cases
sfdc.oppty sfdc_oppty
sfdc_cases
sfdc_oppty.dt


In [26]:
all_variables = []
for e in formatted.split('\n'):
    if sum(list(map(lambda x: '*' in x, re.findall(r"\s[*]?", e)))):
        variable = []
    else:
        variable = [x.strip(' ') for x in re.findall(r"[a-z_\s.]+", e)]
        print(variable)
    all_variables.extend(variable)
    

['sfdc_oppty.dt']
['sfdc_cases.id']
['sfdc.oppty sfdc_oppty']
['', 'sfdc.cases', 'sfdc_cases', 'sfdc_oppty.dt', 'sfdc_cases.dt']
